In [3]:
import boto3

# Get the service resource.
dynamodb = boto3.resource("dynamodb")

# Create the DynamoDB table.
table = dynamodb.create_table(
    TableName="SessionTable",
    KeySchema=[{"AttributeName": "SessionId", "KeyType": "HASH"}],
    AttributeDefinitions=[{"AttributeName": "SessionId", "AttributeType": "S"}],
    BillingMode="PAY_PER_REQUEST",
)

# Wait until the table exists.
table.meta.client.get_waiter("table_exists").wait(TableName="SessionTable")

# Print out some data about the table.
print(table.item_count)

0


In [2]:
from langchain_community.chat_message_histories import (
    DynamoDBChatMessageHistory,
)
history = DynamoDBChatMessageHistory(table_name="SessionTable", session_id="0")

history.add_user_message("hi!")

history.add_ai_message("whats up?")

In [6]:
history.add_ai_message("pra onde vai?")

In [5]:
history.messages

[HumanMessage(content='hi!'),
 AIMessage(content='whats up?'),
 AIMessage(content='pra onde vai?'),
 HumanMessage(content='hi!'),
 AIMessage(content='whats up?')]

In [6]:
dir(history)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 'aadd_messages',
 'aclear',
 'add_ai_message',
 'add_message',
 'add_messages',
 'add_user_message',
 'aget_messages',
 'clear',
 'history_size',
 'key',
 'messages',
 'session_id',
 'table',
 'ttl',
 'ttl_key_name']

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

In [11]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

OPENAI_API_KEY = 'sk-proj-zfM7I0DbwVOSrCqSjsYFT3BlbkFJsCcbfFZaDNxMdYF1DwKn'

chain = prompt | ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [12]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=session_id
    ),
    input_messages_key="question",
    history_messages_key="history",
)

In [13]:
# This is where we configure the session id
config = {"configurable": {"session_id": "35"}}

In [14]:
chain_with_history.invoke({"question": "Hi! I'm bob"}, config=config)

AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 22, 'total_tokens': 32}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e9486bfd-cd85-4ed5-b877-4bdeae5a2eb5-0', usage_metadata={'input_tokens': 22, 'output_tokens': 10, 'total_tokens': 32})

In [17]:
chain_with_history.invoke({"question": "como é o nome da minha amiga?"}, config=config)

AIMessage(content='O nome da sua amiga é Micaelle.', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 103, 'total_tokens': 114}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3c5748f2-99af-4489-acd8-93185bf60da4-0', usage_metadata={'input_tokens': 103, 'output_tokens': 11, 'total_tokens': 114})